<a href="https://colab.research.google.com/github/BBANGSUL/Project/blob/main/Part1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확인하세요!

### 제출 전에 모든 코드가 순차적으로 잘 작동하는지 확인하세요. 
- **런타임 다시 시작**한 다음(메뉴바에서 런타임$\rightarrow$런타임 다시 시작 선택) **모든 셀을 실행**해 보세요(메뉴바에서 런타임$\rightarrow$모두 실행 선택).

### 지정 셀의 지정 영역에서만 코드를 작성했는지 확인하세요.
- `YOUR CODE HERE`이 적혀 있는 셀에 대한 코드만 채점에 반영됩니다.
- `YOUR CODE HERE` ~ `YOUR CODE ENDS HERE` 사이에 본인의 코드를 작성하세요.
- 지정 셀 이외에 본인이 코드 실행 결과를 확인하기 위한 용도로 셀을 추가해도 됩니다.
    - **제출 시 지정 셀에서는 정답 이외에 결과 확인을 위한 코드 등은 제거하고 제출해 주세요!**
    - **기존 셀을 삭제하지 마세요! 채점에 불이익이 있을 수 있습니다.**

### 제출 방법
- 시간 내에 문제를 풀고, 본인의 github repo에 push 하세요.
    - **디렉토리 구조가 변하면 안 됩니다.** `codestates/ds-sc2xx`와 디렉토리 구조가 동일해야 합니다. 답안 파일의 이름을 변경하지 않았는지, 기존에 없던 폴더가 추가되지 않았는지 등을 확인하세요.
    - **코드스테이츠 가입 시 연동한 github 계정을 사용해야 합니다.**
    - **branch를 변경하여 작업하지 마세요.**
        - `master` branch에의 제출 사항에 대해서만 평가합니다.
        - 본인의 github repo `master` branch에 제출이 잘 되어 있는지 확인하세요.

---

# SC22x
 
### 코비 브라이언트의 슛 성공 여부 예측해보기

이번 스프린트 챌린지에서는 故 코비 브라이언트 선수의 NBA 필드 골 시도들이 담긴 데이터셋을 다룹니다 (1996년 11월 3일부터 2016년 4월 13일까지의 정규 시즌과 플레이오프 시즌 기록).

여러분은 각 슛의 성공 여부를 슈팅의 대한 정보, 게임에 대한 정보를 통해서 예측해야 합니다. 이것은 상당히 어려운 예측입니다. 해당 데이터셋은 [NBA API]((https://github.com/swar/nba_api) 파이썬 라이브러리를 통해 수집 됐습니다. 

In [91]:
import sys
import warnings

if "google.colab" in sys.modules:
    ## Colab에서 실행하는 경우 필요한 라이브러리를 미리 설치합니다.
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
# 데이터셋을 불러옵니다
import pandas as pd

url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/nba/nba_kobe_bryant.csv"
df = pd.read_csv(url)

# 데이터셋이 잘 불러졌는지 확인합니다
assert df.shape == (30697, 19)

In [93]:
df.head().T

,0,1,2,3,4
game_id,29600027,29600031,29600044,29600044,29600044
game_event_id,102,127,124,144,151
player_name,Kobe Bryant,Kobe Bryant,Kobe Bryant,Kobe Bryant,Kobe Bryant
period,1,2,2,2,2
minutes_remaining,0,10,8,6,5
seconds_remaining,42,8,37,34,27
action_type,Jump Shot,Jump Shot,Jump Shot,Jump Shot,Jump Shot
shot_type,2PT Field Goal,2PT Field Goal,3PT Field Goal,3PT Field Goal,2PT Field Goal
shot_zone_basic,Mid-Range,Mid-Range,Mid-Range,Restricted Area,In The Paint (Non-RA)
shot_zone_area,Left Side Center(LC),Left Side Center(LC),Left Side Center(LC),Center(C),Center(C)


In [94]:
# 살펴보기
df.isnull().sum()

game_id              0
game_event_id        0
player_name          0
period               0
minutes_remaining    0
seconds_remaining    0
action_type          0
shot_type            0
shot_zone_basic      0
shot_zone_area       0
shot_zone_range      0
shot_distance        0
loc_x                0
loc_y                0
shot_made_flag       0
game_date            0
htm                  0
vtm                  0
season_type          0
dtype: int64

# Part 1 - Data Preprocessing

## 1.1 분류를 위한 베이스라인 정확도를 계산하는 `calculate_baseline_score` 함수를 작성하세요.
- 예측해야 하는 target은 `shot_made_flag` 입니다. 
- 전체 데이터를 써서 baseline **accuracy score**를 계산하세요.
- 최빈 클래스를 사용하는 baseline을 가정하세요.

In [95]:
def calculate_baseline_score(df):
    accuracy_score = 0

    # YOUR CODE HERE
    target = 'shot_made_flag'
    data = df[target]

    fail = len(df[df[target] == 0])
    goal = len(df[df[target] == 1])

    most_frequent = 0
    less_frequent = 0

    if fail > goal:
      most_frequent = fail
      less_frequent = goal

    else :
      most_frequent = goal
      less_frequent = fail


    accuracy_score = most_frequent / (most_frequent + less_frequent)
    # YOUR CODE ENDS HERE

    return accuracy_score

In [96]:
baseline_accuracy_score = calculate_baseline_score(df)

In [97]:
print("baseline accuracy score:", baseline_accuracy_score)

baseline accuracy score: 0.5526924455158485


## 1.2 데이터셋을 아래의 기준을 사용해서 train/test 셋으로 나누는 `split_data` 함수를 작성하세요.

- `test set`: **2014-15** ~ **2015-16** 시즌
- NBA 시즌은 **10월**에 시작해서 **6월**에 종료합니다. 

In [98]:
def split_data(df):
    train = None
    test = None

    # YOUR CODE HERE
    test = df[df['game_date'] >= '2014-10-01']
    train = df[df['game_date'] < '2014-10-01']
    test.reset_index(drop = True, inplace = True)
    # YOUR CODE ENDS HERE

    return train, test

In [99]:
train, test = split_data(df.copy())

## 1.3 새로운 feature를 만드는 `feature_engineering` 함수를 작성하세요.

- 새로운 이름을 갖는 최소 2개 이상의 새로운 feature를 만드세요. 아래의 리스트를 사용하셔도 되고 스스로의 아이디어로 만들어 주셔도 괜찮습니다.
    - **Homecourt Advantage**: 홈 팀(`htm`)이 Los Angeles Lakers (`LAL`)인지 여부
    - **Opponent**: LA Lakers를 상대하는 팀을 나타내는 feature
    - **Seconds remaining in the period**: 피리어드에 남은 총 시간 (분과 초를 합칩니다)
    - **Seconds remaining in the game**: 게임에 남은 총 시간 (피리어드와 초를 합칩니다). 농구에서 게임은 총 4개의 피리어드로 이루어져 있습니다 (각 12분)
    - **Made previous shot**: 코비 브라이언트의 예전 슛이 성공했는지 여부
- 학습에 사용하지 않는 특성은 drop하세요.
    

In [100]:
def feature_engineering(df):
    df_tmp = df.copy()

    # YOUR CODE HERE
    adventage = []
    filter = ['player_name', 'season_type', 'game_id', 'game_event_id' ,'game_date', 'season_type', 'period' ,'seconds_remaining', 'htm', 'vtm']

    for i in range(len(df_tmp)):
      if df_tmp['htm'][i] == 'LAL':
        adventage.append(1)

      else:
        adventage.append(0)

    df_tmp['Homecourt Advantage'] = adventage
    df_tmp['Seconds remaining in the period'] = df_tmp['seconds_remaining'] + df_tmp['minutes_remaining'] * 60
    df_tmp['Distance'] = (df_tmp['loc_x'] ** 2 + df_tmp['loc_y'] ** 2) ** 0.5
    df_tmp = df_tmp.drop(filter, axis = 1)

    # YOUR CODE ENDS HERE

    return df_tmp

In [101]:
train_processed = feature_engineering(train)
test_processed = feature_engineering(test)

In [102]:
train_processed.head()

,minutes_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,Homecourt Advantage,Seconds remaining in the period,Distance
0,0,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1,42,181.813091
1,10,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,0,608,163.003067
2,8,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,0,517,230.054341
3,6,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,0,394,0.000000
4,5,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,0,327,138.361844


In [103]:
test_processed.head()

,minutes_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,Homecourt Advantage,Seconds remaining in the period,Distance
0,6,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,-54,201,0,1,373,208.127365
1,4,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-16,63,1,1,285,65.000000
2,5,Turnaround Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,166,78,0,1,352,183.412104
3,9,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-138,241,0,1,576,277.713882
4,7,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-179,88,0,1,436,199.461776


## 1.4 train / test 셋을 features(X)와 target(y)으로 각각 나누는 `divide_data` 함수를 작성하세요.
- `pd.DataFrame` 형태의 `X`와 `pd.Series` 형태의 `y`를 반환하세요.

In [104]:
def divide_data(df):
    X = None
    y = None

    # YOUR CODE HERE
    target = 'shot_made_flag'

    X = df.drop(target, axis = 1)
    y = pd.Series(df[target])
    # YOUR CODE ENDS HERE

    return X, y

In [105]:
X_train, y_train = divide_data(train_processed)
X_test, y_test = divide_data(test_processed)

In [106]:
X_train.shape, X_test.shape

((28871, 12), (1826, 12))

# Part 2 - Modeling & Evaluation

## 2.1 Decision Tree 또는 RandomForest 모델 학습 파이프라인을 생성하고 학습하는 `fit_trees` 함수를 작성하세요. 
- Scikit-learn의 `pipeline`을 활용하여 범주형 feature의 encoder, imputer, 모델을 포함하는 하나의 학습 파이프라인을 구축하세요.
- 구축한 파이프라인에 대해 `RandomizedSearchCV`를 사용해 CV를 진행하세요.
    - 적어도 두 개 이상의 하이퍼파라미터에 대해 탐색을 진행하세요.
    - F1 score을 최대화하는 하이퍼파라미터 조합을 선택하세요.
- RandomForestClassifier, RandomizedSearchCV 등 랜덤성이 포함되는 방법론에 대해 모두 `random_state=42`로 하세요.
- cv는 5 이하, n_iter은 10회 이하로 하세요.

In [107]:
y_train.value_counts(normalize = True)

0    0.547401
1    0.452599
Name: shot_made_flag, dtype: float64

In [108]:
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder, OneHotEncoder

### IMPORT ANY
# YOUR CODE HERE
from scipy.stats import randint, uniform

# YOUR CODE ENDS HERE


def fit_trees(X_train, y_train):
    pipe = None    # pipe = make_pipeline(...)
    clf = None     # clf = RandomizedSearchCV(pipe, ...)

    # YOUR CODE HERE
    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(),
        RandomForestClassifier(random_state = 42, 
                               n_estimators = 300,
                               n_jobs = -1)
    )

    dists = {
        'randomforestclassifier__max_depth' : randint(1, 4),
        'randomforestclassifier__class_weight' : [None, 'balanced']
    }

    clf = RandomizedSearchCV(
        pipe,
        param_distributions = dists,
        n_iter = 5,
        cv = 5,
        scoring = 'f1',
        random_state = 42
    )
    
    clf.fit(X_train, y_train)
    # YOUR CODE ENDS HERE

    return clf

In [109]:
clf = fit_trees(X_train, y_train)

In [110]:
print("Best CV F1", clf.best_score_)
print("Best CV params", clf.best_params_)

Best CV F1 0.5902627240807871
Best CV params {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 1}


## 2.2 학습된 모델로부터 feature importance를 계산하는 `get_feature_importances` 함수를 작성하세요. 
- feature_importances 변수에 RandomForest / Decision Tree의 feature_importances_ 값을 할당하면 됩니다.

In [111]:
import numpy as np


def get_feature_importances(pipe, feature_names):
    feature_importances = np.zeros(len(feature_names))

    # YOUR CODE HERE
    feature_importances = pipe.named_steps['randomforestclassifier'].feature_importances_    
    # YOUR CODE ENDS HERE

    return sorted(
        list(zip(feature_names, feature_importances)), key=lambda x: x[1], reverse=True
    )

In [112]:
feature_importances = get_feature_importances(
    clf.best_estimator_, list(X_train.columns)
)

In [113]:
for name, importance in feature_importances:
    print(f"{name}: {importance:.4f}")

action_type: 0.2500
Distance: 0.1733
loc_y: 0.1367
shot_distance: 0.1333
shot_zone_range: 0.0967
shot_type: 0.0900
shot_zone_basic: 0.0600
shot_zone_area: 0.0400
loc_x: 0.0133
Seconds remaining in the period: 0.0067
minutes_remaining: 0.0000
Homecourt Advantage: 0.0000


## 2.3 본인이 만든 모델의 Test 셋에서의 성능을 확인해 보세요.
- ### test_accuracy > 0.6, test_f1 > 0.5를 만족하면 기본 성능점수를 얻습니다.
- ###  (도전과제) test_accuracy > 0.68, test_f1 > 0.54을 만족하면 성능점수 만점을 얻습니다.
  - 도전과제까지 수행해야 스챌 3점을 얻습니다. 
  - 1.3으로 돌아가 다양한 특성 조합을 시도해 보세요. 
    - 그리고 2.2로 돌아와 피쳐 중요도를 계산해 보세요.
    - 어떤 피쳐의 중요도가 가장 높나요?
    - 중요도가 낮은 피쳐라도 있는 게 좋을까요?
  - 2.1로 돌아가 SearchCV에서 다양한 하이퍼파라미터 옵션 조합을 선택해 보세요.
    - 본인이 선택한 모델에서, 각 하이퍼파라미터가 어떤 의미를 가지나요?
    - SearchCV가 더 다양한 하이퍼파라미터 조합을 시도하도록 하기 위해 SearchCV의 어떤 옵션을 조정해야 할까요?

In [114]:
def calculate_score(pipe, X_test, y_test):
    test_accuracy = pipe.score(X_test, y_test)
    test_f1 = f1_score(y_test, pipe.predict(X_test))
    return test_accuracy, test_f1

In [115]:
test_accuracy, test_f1 = calculate_score(clf.best_estimator_, X_test, y_test)

In [116]:
print("test accuracy: ", test_accuracy)
print("test F1:", test_f1)

test accuracy:  0.6659364731653888
test F1: 0.5566860465116279


In [117]:
print("test accuracy > 0.68?: ", test_accuracy > 0.68)
print("test F1 > 0.54?:", test_f1 > 0.54)


test accuracy > 0.68?:  False
test F1 > 0.54?: True


---

# 수고하셨습니다!